In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import when
from pyspark.sql.functions import col
from pyspark.sql.types import * 
from pyspark.sql.functions import lit
import pyodbc
import pandas as pd
import pyspark.sql.functions as sf

### Using MySQL

In [3]:
# Create SparkSession for MySQL
spark = SparkSession.builder.appName("test").config('spark.driver.extraClassPath', '/opt/homebrew/Cellar/apache-spark/3.5.0/libexec/jars/mysql-connector-j-8.2.0.jar').getOrCreate()

24/02/17 15:37:53 WARN Utils: Your hostname, Marcus-MacBook.local resolves to a loopback address: 127.0.0.1; using 192.168.1.4 instead (on interface en0)
24/02/17 15:37:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/17 15:37:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# MySQL configurations
database = 'YOUR_DATABASE_NAME'
url = f'jdbc:mysql://localhost/{database}'
driver = 'com.mysql.cj.jdbc.Driver' # New driver for MySQL
dbtable = 'YOUR_TABLE_NAME'
user = 'YOUR_USER_NAME' # Default user: root
password = 'YOUR_PASSWORD'

In [5]:
# Read from MySQL
df = spark.read.format('jdbc').option('url', url) \
    .option('driver', driver) \
    .option('dbtable', dbtable).option('user', user).option('password', password).load()

In [6]:
df.show()

+---------+-------+-----------+-----------+------------------+---------+---+----------+-----------+-------------+
|departure|arrival|depart_time|arrive_time|           company|    price| id|date_leave|date_return|scraping_date|
+---------+-------+-----------+-----------+------------------+---------+---+----------+-----------+-------------+
|      HCM|    HAN|      21:00|      23:10|       VietJet Air|6,180,000|  1|2024-02-06| 2024-02-16|   2023-12-25|
|      HCM|    HAN|      21:00|      23:10|       VietJet Air|6,352,800|  2|2024-02-05| 2024-02-17|   2023-12-25|
|      HCM|    HAN|      05:00|      07:05|  Vietnam Airlines|7,050,000|  3|2024-02-06| 2024-02-18|   2023-12-25|
|      HCM|    HAN|      12:10|      02:20|       VietJet Air|6,007,200|  4|2024-02-05| 2024-02-16|   2023-12-25|
|      HCM|    HAN|      22:50|      01:00|       VietJet Air|6,180,000|  5|2024-02-06| 2024-02-16|   2023-12-25|
|      HCM|    HAN|      21:40|      23:50|       VietJet Air|6,352,800|  6|2024-02-05| 

In [13]:
# Write to MySQL
df.write.format('jdbc').option('url',url).option('driver',driver).option('dbtable','customer_behaviour').option('user',user).option('password',password).mode('append').save()

### Using AzureSQL

In [3]:
# Create SparkSession for AzureSQL
spark = SparkSession.builder.config('spark.jars.packages','com.microsoft.azure:spark-mssql-connector:1.0.2').getOrCreate()

24/02/27 09:06:07 WARN Utils: Your hostname, Marcus-MacBook.local resolves to a loopback address: 127.0.0.1; using 192.168.1.6 instead (on interface en0)
24/02/27 09:06:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/marcusle02/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/marcusle02/.ivy2/cache
The jars for the packages stored in: /Users/marcusle02/.ivy2/jars
com.microsoft.azure#spark-mssql-connector added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-64e05673-39a6-4c0a-8cb2-ff3e02075d2c;1.0
	confs: [default]
	found com.microsoft.azure#spark-mssql-connector;1.0.2 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
:: resolution report :: resolve 95ms :: artifacts dl 3ms
	:: modules in use:
	com.microsoft.azure#spark-mssql-connector;1.0.2 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0 

In [4]:
# AzureSQL configurations
jdbcHostname = "YOUR_HOST_NAME"
jdbcDatabase = "YOUR_DATABASE"
jdbcPort = "1433"
username = "YOUR_USER"
password = "YOUR_PASSWORD"
jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)

connectionProperties = {
"user" : username,
"password" : password,
"driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [5]:
pushdown_query = """(SELECT * FROM SalesLT.DRM_Data) A"""

In [ ]:
# Read from AzureSQL
df = spark.read.format("com.microsoft.sqlserver.jdbc.spark").option("url","jdbc:sqlserver://longth.database.windows.net:1433;database=LongTH").\
    option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver").\
        option("dbtable",pushdown_query).option("user",username).\
            option("password",password).load()

In [15]:
df.show()

+---------+----------+------------+
|Total_Key|      Date|Service_Name|
+---------+----------+------------+
|       79|2019-08-02|         BHD|
|     3513|2019-08-02|        Fim+|
|     8816|2019-08-02|     PhimGoi|
|       77|2019-08-03|         BHD|
|     4039|2019-08-03|        Fim+|
|     9294|2019-08-03|     PhimGoi|
|       81|2019-08-04|         BHD|
|     3963|2019-08-04|        Fim+|
|     9460|2019-08-04|     PhimGoi|
|       68|2019-08-05|         BHD|
|     3194|2019-08-05|        Fim+|
|     9541|2019-08-05|     PhimGoi|
|       85|2019-08-06|         BHD|
|     3541|2019-08-06|        Fim+|
|     9793|2019-08-06|     PhimGoi|
|       78|2019-08-07|         BHD|
|     3441|2019-08-07|        Fim+|
|    10958|2019-08-07|     PhimGoi|
|       82|2019-08-08|         BHD|
|     3566|2019-08-08|        Fim+|
+---------+----------+------------+
only showing top 20 rows

